In [14]:
import numpy as np
import pandas as pd
import cvxpy as cp
from math import ceil

In [15]:
requirements = pd.read_excel('data.xlsx', header=None).values.flatten()
TOTAL_WEEK = 104
WASTE_RATE = 0.2
K = 10
wasted_robots = np.array([
    round(x * WASTE_RATE) if x % 1 != 0.5 else ceil(4 * x * WASTE_RATE) 
    for x in requirements
])
requirements ,wasted_robots

(array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
          5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
          9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
         10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
         50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
         41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
         54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
         89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
       dtype=int64),
 array([ 2,  1,  1,  1,  3,  1,  1,  1,  3,  1,  1,  1,  2,  1,  1,  1,  2,
         1,  1,  2,  6,  4,  3,  4,  5,  3,  2,  2,  3,  1,  1,  1,  2,  1,
         1,  1,  2,  1,  1,  2,  3,  2,  2,  2,  3,  2,  2,  3,  5,  4,  5,
         7, 10,  9,  9, 10, 11,  9,  8,  9, 10,  9,  8,  9, 10,  8,  8,  8,
        10,  7,  7,  7,  8,  7,  7,  9, 11, 10, 11, 13, 16, 14, 15, 17, 20,
        18, 18

In [16]:
working_hands = cp.Parameter(TOTAL_WEEK)
working_hands.value = 4 * requirements
wasted_hands = cp.Parameter(TOTAL_WEEK)
wasted_hands.value = 4 * wasted_robots
working_bodies = cp.Parameter(TOTAL_WEEK)
working_bodies.value = requirements
wasted_bodies = cp.Parameter(TOTAL_WEEK)
wasted_bodies.value = wasted_robots

new_hands = cp.Variable(TOTAL_WEEK, integer=True)
teaching_hands = cp.Variable(TOTAL_WEEK, integer=True)
resting_hands = cp.Variable(TOTAL_WEEK, integer=True)
new_bodies = cp.Variable(TOTAL_WEEK, integer=True)
resting_bodies = cp.Variable(TOTAL_WEEK, integer=True)

In [17]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == 50,
    resting_bodies[0] + working_bodies[0] == 13,
    K * teaching_hands >= new_hands,
    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0,
    resting_bodies >= 0,
    new_bodies >= 0
]

for t in range(TOTAL_WEEK - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - wasted_hands[t]),
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + new_hands[t] - wasted_hands[t]
        )
    cons.append(resting_bodies[t + 1] + working_bodies[t + 1] == \
        resting_bodies[t] + working_bodies[t] + new_bodies[t] - wasted_bodies[t]    
    )

In [18]:
obj = cp.Minimize(
    100 * cp.sum(new_hands)\
    + 200 * cp.sum(new_bodies)
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(resting_bodies)\
    + 10 * cp.sum(teaching_hands + new_hands)
)

In [19]:
prob = cp.Problem(obj, cons)
prob.solve()

669479.999999995

In [20]:
new_bodies.value

array([ 0., -0., -0.,  8., -0., -0., -0.,  3., -0., -0., -0.,  5., -0.,
       -0., -0.,  2., -0., -0.,  6., 20., -0., -0.,  4., 11., -0., -0.,
       -0.,  1., -0., -0., -0.,  1., -0., -0., -0.,  6., -0., -0.,  2.,
        7., -0., -0.,  3.,  6., -0., -0.,  8., 13., -0.,  6., 18., 21.,
        5.,  9., 13., 18., -0.,  3., 12., 17.,  1.,  8., 11., 16., -0.,
        5., 10., 15., -0.,  3.,  8., 14., -0.,  9., 14., 21.,  4., 16.,
       22., 28.,  6., 18., 26., 33.,  8., 17., 20., 28.,  7., 18., 19.,
       33.,  6., 18., 19., 34., 11., 17., 24., 30., 12., 17., 25., -0.])